In [1]:
# %matplotlib notebook
import NN_Module as nnm
import torch
import numpy as np
import numpy.ma as ma
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import math
import time
import pandas as pd
from pandas import read_csv

In [2]:
# Functions

# Target function to approximate
def target_func(input):
    """
    This is the fucntion the network will try to approximate.

    Inputs: input (Pytorch tensor)

    Outputs: output (Pytorch tensor)
    """
    return torch.unsqueeze(input[:,0]**2 * torch.cos(input[:,1]), 1)

In [3]:
# Hyperparameters
N = 2
train_size = 1000000
test_size = 10000
hidden_nodes = 1000
hidden_layers = 2
batch_size = 100
n_epochs = 500
learning_rate = 1e-4
lr_red_factor = 0.2
lr_red_patience = 40
lr_red_threshold = 1e-2
weight_decay = 0

In [4]:
# Create a list of sets of N random numbers, the training set
inputs = (100 * torch.rand((train_size,N),dtype=torch.float32) - 50).cuda()
outputs = target_func(inputs)

In [5]:
# create a test set
test_inputs = (100 * torch.rand((test_size,N),dtype=torch.float32) - 50).cuda() # Uniform
# test_inputs = torch.randn((test_size,N),dtype=torch.float32).cuda() # Normal
test_outputs = target_func(test_inputs)

In [6]:
# Standardize data sets
input_stats = nnm.find_stats(inputs)
output_stats = nnm.find_stats(outputs)
std_inputs = nnm.affine_transform(inputs, input_stats)
std_test_inputs = nnm.affine_transform(test_inputs, input_stats) # Not actually normal; Only std_inputs is normal
std_outputs = nnm.affine_transform(outputs, output_stats)
std_test_outputs = nnm.affine_transform(test_outputs, output_stats)

In [7]:
# Don't standardize
'''
input_stats = (0, 1)
output_stats = (0, 1)
std_inputs = inputs
std_test_inputs = test_inputs
std_outputs = outputs
std_test_outputs = test_outputs
'''

'\ninput_stats = (0, 1)\noutput_stats = (0, 1)\nstd_inputs = inputs\nstd_test_inputs = test_inputs\nstd_outputs = outputs\nstd_test_outputs = test_outputs\n'

In [8]:
# Create a model
model = nnm.create_model(inputs, outputs, hidden_nodes, hidden_layers)

In [ ]:
# Train the model
graph_data = nnm.train_network(model, std_inputs, std_outputs, std_test_inputs, std_test_outputs, output_stats, batch_size, n_epochs, learning_rate, lr_red_factor, lr_red_patience, lr_red_threshold, weight_decay)

=>Starting 1/500 epochs.
=>Starting 11/500 epochs.
=>Starting 21/500 epochs.
=>Starting 31/500 epochs.
=>Starting 41/500 epochs.
=>Starting 51/500 epochs.
=>Starting 61/500 epochs.
=>Starting 71/500 epochs.
=>Starting 81/500 epochs.
=>Starting 91/500 epochs.
=>Starting 101/500 epochs.
=>Starting 111/500 epochs.
=>Starting 121/500 epochs.
=>Starting 131/500 epochs.
=>Starting 141/500 epochs.
=>Starting 161/500 epochs.
=>Starting 171/500 epochs.
=>Starting 181/500 epochs.
=>Starting 191/500 epochs.
=>Starting 201/500 epochs.
=>Starting 211/500 epochs.


In [ ]:
# Graphing
graphs = nnm.new_graphs()
nnm.graphing(graphs, graph_data, test_size, n_epochs, 100, 100)

In [ ]:
# Show graphs
nnm.show_graphs(graphs)

In [ ]:
# Save all graphs
nnm.save_graphs(graphs, 'x^2cos(y).pdf')

In [10]:
# Save graphs
graphs['fig_loss'].savefig('loss.pdf')
graphs['fig_accu'].savefig('accu.pdf')
graphs['fig_accu_out'].savefig('accu_out.pdf')
graphs['fig_out_residual'].savefig('out_residual.pdf')
graphs['fig_histograms'].savefig('histograms.pdf')

In [7]:
# Analysis
param_list = [[1000, 1, 500, 1e-4, 0.2, 40, 1e-2, 0], [5000, 1, 500, 1e-4, 0.2, 40, 1e-2, 0], [1000, 2, 500, 1e-4, 0.2, 40, 1e-2, 0], [5000, 2, 500, 1e-4, 0.2, 40, 1e-2, 0]] # [nodes, layers, epochs, learning rate, lr_red_factor, lr_red_patience, lr_red_threshold, weight decay]
trials = 1 # The number of trials for each configuration
analysis_data = nnm.analyze(param_list, trials, inputs, outputs, test_inputs, test_outputs, output_stats, batch_size)

Training done!


KeyboardInterrupt: 

In [ ]:
# Graphing
analysis_graphs = nnm.new_analysis_graphs()
nnm.analysis_graphing(analysis_graphs, analysis_data, param_list, trials)

In [ ]:
analysis_graphs['fig_analysis'].savefig('analysis.pdf')

In [22]:
# Send data to be displayed
df = pd.DataFrame(data=analysis_data)

In [23]:
# Display analysis data
### Make graphs
print(df)

     nodes  layers  epochs      time  accuracy
0    100.0     1.0     0.0  0.024497     0.001
1    100.0     1.0     1.0  0.192594     0.002
2    100.0     1.0     2.0  0.347872     0.004
3    100.0     1.0     3.0  0.503090     0.008
4    100.0     1.0     4.0  0.664804     0.015
..     ...     ...     ...       ...       ...
445  100.0     1.0    45.0  7.133812     0.063
446  100.0     1.0    46.0  7.292924     0.070
447  100.0     1.0    47.0  7.449513     0.076
448  100.0     1.0    48.0  7.604756     0.079
449  100.0     1.0    49.0  7.766390     0.081

[450 rows x 5 columns]


In [17]:
# Uploads existing data and adds it to working dataset
# file_upload = read_csv("NN_Test_Data_1.csv")
# print(file_upload)
# df = df.append(file_upload)

In [24]:
# Saves run data as csv file
df.to_csv("NN_Test_Data_1.csv", index=False)